<a href="https://colab.research.google.com/github/hardiksiloiya/GANs/blob/main/simple_autoencoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [124]:
import sys
import os
import keras
from keras.models import Model
import numpy as np
from keras.layers import Input
from tensorflow.keras.layers import Conv2D,Conv2DTranspose
from keras.datasets import mnist
from tensorflow.keras.layers import BatchNormalization
from keras.layers import LeakyReLU
from keras.layers import Dense, Activation, Flatten, Dropout,Reshape
from keras.optimizers import Adam
from keras import backend as K



In [125]:
(x_train,y_train),(x_test,y_test)=mnist.load_data()
x_train=x_train.reshape(60000,28,28,1)
x_test=x_test.reshape(10000,28,28,1)
x_train = x_train.astype(np.float32)
y_train = y_train.astype(np.float32)
x_test=x_test.astype(np.float32)
y_test=y_test.astype(np.float32)

In [142]:
input_layer=Input((28,28,1))
x=Conv2D(filters=30,kernel_size=3,strides=1,padding='same')(input_layer)
x=BatchNormalization()(x)
x=LeakyReLU()(x)

x=Conv2D(filters=10,kernel_size=3,strides=1,padding='same')(x)
x=BatchNormalization()(x)
x=LeakyReLU()(x)

x=Conv2D(filters=15,kernel_size=3,strides=1,padding='same')(x)
x=LeakyReLU()(x)
x=Dropout(rate=0.25)(x)

x=Conv2D(filters=20,kernel_size=3,strides=1,padding='same')(x)
x=BatchNormalization()(x)
x=LeakyReLU()(x)
x=Flatten()(x)


x=Dense(2)(x)
out1=x

encode=Model(input_layer,out1)
#encode.summary()

In [143]:
input_layer2=Input(shape=2)
y=Dense(1568)(input_layer2)
y=Reshape((7,7,32))(y)
y=Conv2DTranspose(filters = 15, kernel_size = 3, strides = 2, padding = 'same')(y)
y=Conv2DTranspose(filters=30,kernel_size=3,strides=2,padding='same')(y)
y=Conv2DTranspose(filters=20,kernel_size=3,strides=1,padding='same')(y)

y=Conv2DTranspose(filters=1,kernel_size=3,strides=1,padding='same')(y)
y=Activation('relu')(y)
out2=y

decode=Model(input_layer2,out2)
#decode.summary()


In [144]:
model=Model(input_layer,decode(x))
model.summary()

Model: "model_37"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_51 (InputLayer)        [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d_59 (Conv2D)           (None, 28, 28, 30)        300       
_________________________________________________________________
batch_normalization_73 (Batc (None, 28, 28, 30)        120       
_________________________________________________________________
leaky_re_lu_68 (LeakyReLU)   (None, 28, 28, 30)        0         
_________________________________________________________________
conv2d_60 (Conv2D)           (None, 28, 28, 10)        2710      
_________________________________________________________________
batch_normalization_74 (Batc (None, 28, 28, 10)        40        
_________________________________________________________________
leaky_re_lu_69 (LeakyReLU)   (None, 28, 28, 10)        0  

In [145]:
optimizer=Adam(lr=0.0005)
def loss(a,b):
 return K.mean(K.square(a-b), axis = [1,2,3])
 
model.compile(optimizer=optimizer,loss=loss)

In [ ]:
model.fit(x=x_train,y=x_train,batch_size=32,shuffle=True,epochs=10)

Epoch 1/10
 314/1875 [====>.........................] - ETA: 4:48 - loss: 5107.2920